# This notebook requires a deployment named "gpt-4o", edit as needed

## Web Search Ant As Tool

This example demonstrates one ant using another ant as a tool so that the second ant's work is outside the main thread.

The **WebSearchAgentAnt** demonstrates the use of a Web API secured using an API key with an auth header. 
This sample requires you to create a Web Search (bing) resource in Azure and enter its corresponding API key in [0-AI-settings](0-AI-settings.ipynb)

WebSearchAgentAnt uses **WebPageAgentAnt** via a tool call to evaluate search results using the search results link.

This demo uses a lot of tokens. 

**If your quota is low, you will get throttled**

## First steps
- First, download AntRunLib from Nuget
- At least once, set up the environment using **[0-AI-settings](0-AI-settings.ipynb)**

In [1]:
#r "nuget: AntRunnerLib, 0.9.5"

using AntRunnerLib;
using AntRunnerLib.Identity;
using static AntRunnerLib.ClientUtility;
using System.IO;

#!import config/Settings.cs 

var envVariables = Settings.GetEnvironmentVariables();
foreach (var kvp in envVariables)
{
    Environment.SetEnvironmentVariable(kvp.Key, kvp.Value);
}

Environment.SetEnvironmentVariable("AZURE_OPENAI_DEPLOYMENT", "gpt-4o");

var config = AzureOpenAiConfigFactory.Get();
var client = GetOpenAiClient(config);


Installed Packages AntRunnerLib, 0.9.5

In [2]:
var assistantId = await AssistantUtility.Create("WebSearchAgentAnt", config);
Console.WriteLine(assistantId);
assistantId = await AssistantUtility.Create("WebPageAgentAnt", config);
Console.WriteLine(assistantId);

asst_Z8StODtvI35Re12fF5EofYDh
asst_3hJpYkI52tZTpRX4ixEyIgsG


In [3]:
AntLoader.LoadAssembly(@".\AssistantDefinitions\WebPageAgentAnt\HtmlAgilityPack.dll");
AntLoader.LoadAssembly(@".\AssistantDefinitions\WebPageAgentAnt\HtmlAgilityPackExtensions.dll");

## Run the Assistant

`output.Dialog` shows the conversation.

In [4]:
var assistantRunOptions = new AssistantRunOptions() {
    AssistantName = "WebSearchAgentAnt",
    Instructions = "Use python to get todays date and then use it to find things to do in Atlanta today. Be careful to not include events in the past in your final answer.",
};
var output = await AntRunnerLib.AssistantRunner.RunThread(assistantRunOptions, config);
output.Dialog.DisplayAs("text/markdown");


User: Use python to get todays date and then use it to find things to do in Atlanta today. Be careful to not include events in the past in your final answer.

Assistant: I ran this code:
```
from datetime import datetime

# Get today's date
today_date = datetime.now().strftime("%Y-%m-%d")
today_date
```

I called the tool named crawl with these arguments:
```
{"q":"things to do in Atlanta December 31 2024","textFormat":"Raw"}
```
and got this result:
```
{"queryContext":{"originalQuery":"things to do in Atlanta December 31 2024"},"webPages":{"value":[{"name":"Atlanta Events in December 2024 - Schedule \u0026 Tickets","url":"https://atlanta-ga.events/december/","displayUrl":"https://atlanta-ga.events/december","snippet":"Check Out The Atlanta Events Calendar for December 2024: Current \u0026 Upcoming Events \u002B Best \u0026 Fun Thing To Do in the City. ... Dec 31, 2024 3:30 AM . Georgia Tech Yellow Jackets Basketball . Events: 3. McCamish Pavilion . Dec 31, 2024 2:30 PM ..."},{"name":"11 Atlanta holiday events happening this weekend | 11alive.com","url":"https://www.11alive.com/article/entertainment/events/11-things-to-do-in-atlanta-dec-26-31/85-46d65687-9cd6-4513-9fcb-5e2012452a26","displayUrl":"https://www.11alive.com/article/entertainment/events/11-things-to-do-in-atlanta-dec-26...","snippet":"Updated: 7:20 AM EST December 26, 2024 ATLANTA \u2014 The holiday season is in full swing, and Atlanta is buzzing with exciting events to enjoy this weekend! ... Date and Location: Dec. 25\u201331; 10 a ..."},{"name":"Atlanta Events - December 2024 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-december.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-december.html","snippet":"Things to do in Atlanta, GA in December, including upcoming festivals, Christmas events and other holiday celebrations, lights shows, holiday markets, comedy, musicals, ballet, tours of homes, and more. ... December 31 2024 New Year\u2019s eve event at the Georgia Aquarium, with DJ and dancing, food, and after-hours access to the aquatic exhibits.","thumbnailUrl":"https://www.bing.com/th?id=OIP.xm-3dn3hYJZHsVVU9WO5qwHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Events \u0026 Things To Do in Atlanta in December 2024 - AllEvents.in","url":"https://allevents.in/atlanta/december","displayUrl":"https://allevents.in/atlanta/december","snippet":"Find events and things to do in december 2024 in Atlanta. Discover parties, concerts, meets,shows, sports, club, reunion, Performance happening in 2024 in Atlanta. Open Menu. Atlanta Change City; ... Tue, 31 Dec Trending events in december in Atlanta. Atlanta Symphony Orchestra: New Year\u0027s Eve"},{"name":"Events \u0026 Things to Do for Christmas in Atlanta - Discover Atlanta","url":"https://discoveratlanta.com/events/seasonal/christmas/","displayUrl":"https://discoveratlanta.com/events/seasonal/christmas","snippet":"The possibilities are endless with so many things to do in Atlanta at Christmas! ... Atlanta Ballet\u2019s The Nutcracker returns this December at Cobb Energy Centre with a larger-than-life production of a winter wonderland and dream-like world of ... 12/31/24. Candlelight Concerts 2024. Pets. 12/30/24 \u2013 1/1/25. Adopt Any Senior Pet for FREE in ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.t6GjYUDzSex1Myxpnsk1ngHaE6\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"December 31 events in Atlanta, GA - Eventbrite","url":"https://www.eventbrite.com/d/ga--atlanta/december-31/","displayUrl":"https://www.eventbrite.com/d/ga--atlanta/december-31","snippet":"HUGEL NEW YEARS EVE | Make The Girls Dance 2024 | District Atlanta. Tue, Dec 31, 10:00 PM. District Atlanta. Save this event: HUGEL NEW YEARS EVE ... TUESDAY DECEMBER 31 2024 Save this event: NYE at EMBR LOUNGE| 8PM-3AM| TUESDAY DECEMBER 31 2024. THE LEMON DROP - NEW YEARS EVE 2K24. Tue, Dec 31, 9:00 PM. Westside Cultural Arts Center."},{"name":"Explore Atlanta Events for Today, This Week or Weekend","url":"https://discoveratlanta.com/events/main/","displayUrl":"https://discoveratlanta.com/events","snippet":"Explore the Atlanta events calendar for things to do today, this week or this weekend. Find upcoming festivals, seasonal events and more! ... New Year\u2019s Eve Celebration at Red Phone Booth 2024 12/31/24. New Year\u2019s Eve Dinner at Signia by Hilton Atlanta 12/31/24 \u2013 1/1/25 ... December 31. Tuesday. Atlanta\u2019s New Year\u2019s Eve Peach Drop ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.FkxSV6JbrWatsFMM3_8dIAHaEh\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta Events Calendar 2024/2025 | Atlanta, GA","url":"https://atlanta-ga.events/","displayUrl":"https://atlanta-ga.events","snippet":"This calendar is your ultimate guide to the best things to do in Atlanta, GA: ... Dec 31, 2024 3:30 AM . Notre Dame Fighting Irish . Events: 3. McCamish Pavilion . Dec 31, 2024 2:30 PM . Georgia Tech Yellow Jackets Basketball ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.ytkv04WKv8RTyYo4_DsP4gHaHa\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"20 free things to do in metro Atlanta in December","url":"https://www.accessatlanta.com/atlanta-things-to-do/20-free-things-to-do-in-metro-atlanta-in-december/RUFTV7LQ2JEGZGUNGNR6FHEARM/","displayUrl":"https://www.accessatlanta.com/atlanta-things-to-do/20-free-things-to-do-in-metro...","snippet":"Here are 20 of the best things to do in Atlanta in December that are totally free. ... and nightly display until Tuesday, Dec. 31; Time: 4 - 8 ... 14, 21, 23, 27, 29 and 31) Holiday Spirit 2024 ..."},{"name":"Cool Things To Do in December in Atlanta","url":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-december-in-atlanta/","displayUrl":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-december-in-atlanta","snippet":"Cool Things To Do in December in Atlanta. By Mary Welch. Published November 14, 2023 Last Updated December 2, 2024. December is a great time to be in Atlanta. The list of cool things to do seems endless. There\u2019s unparalleled shopping, great themed entertainment, holiday light displays galore and wonderful restaurants where you can enjoy a ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.B7WMsn5icdXhjRnlj2WcZAHaHa\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"}]}}
```

I called the tool named ExtractContentFromUrl with these arguments:
```
{"assistantName": "WebPageAgentAnt", "instructions": "Find events in Atlanta on December 31, 2024. https://atlanta-ga.events/december/"}
```
and got this result:
```
"Here are some events happening in Atlanta on December 31, 2024:\n\n1. **Georgia Tech Yellow Jackets vs. Notre Dame Fighting Irish**\n   - **Time**: 2:30 PM\n   - **Venue**: McCamish Pavilion, 965 Fowler St., Atlanta, GA\n   - **Prices**: From $18 (Avg. price ~$62)\n   - **Tickets**: [Get Tickets](https://atlanta-ga.events/event/georgia-tech-yellow-jackets-vs-notre-dame-fighting-irish-atlanta-tuesday-december-31-2024-230-pm/?pid=1282)\n\n   ![Georgia Tech Yellow Jackets Basketball](https://atlanta-ga.events/wp-content/uploads/2023/08/Georgia-Tech-Yellow-Jackets-Basketball.jpg)\n\n2. **Provenance NYE: Seven Lions, Knife Party \u0026 Grabbitz**\n   - **Time**: 7:00 PM\n   - **Venue**: Atlanta Coliseum, 2075 Market St, Duluth, GA\n   - **Prices**: From $114 (Avg. price ~$132)\n   - **Tickets**: [Get Tickets](https://atlanta-ga.events/event/provenance-nye-seven-lions-knife-party-and-grabbitz-duluth-tuesday-december-31-2024-700-pm/?pid=8320)\n\n   ![Provenance NYE](https://atlanta-ga.events/wp-content/uploads/2023/03/Seven-Lions.jpg)\n\n3. **Zac Brown Band**\n   - **Time**: 8:00 PM\n   - **Venue**: State Farm Arena, One Philips Drive, Atlanta, GA\n   - **Prices**: From $73 (Avg. price ~$237)\n   - **Tickets**: [Get Tickets](https://atlanta-ga.events/event/zac-brown-band-atlanta-tuesday-december-31-2024-800-pm/?pid=769)\n\n   ![Zac Brown Band](https://atlanta-ga.events/wp-content/uploads/2024/02/zac-brown-band.jpg)\n\n4. **Moon Taxi**\n   - **Time**: 8:00 PM\n   - **Venue**: The Tabernacle, 152 Luckie St, Atlanta, GA\n   - **Prices**: From $65 (Avg. price ~$127)\n   - **Tickets**: [Get Tickets](https://atlanta-ga.events/event/moon-taxi-atlanta-tuesday-december-31-2024-800-pm/?pid=7640)\n\n   ![Moon Taxi](https://atlanta-ga.events/wp-content/uploads/2024/09/moon-taxi-concert-tickets.jpeg)\n\n5. **Juelz - Artist**\n   - **Time**: 9:00 PM\n   - **Venue**: Believe Music Hall, 181 Ralph David Abernathy Blvd, Atlanta, GA\n   - **Prices**: From $69 (Avg. price ~$79)\n   - **Tickets**: [Get Tickets](https://atlanta-ga.events/event/juelz-artist-atlanta-tuesday-december-31-2024-900-pm/?pid=4969)\n\n6. **Gimme Gimme Disco**\n   - **Time**: 9:00 PM\n   - **Venue**: Hell at The Masquerade, 75 MLK Jr Dr SW, Atlanta, GA\n   - **Prices**: From $49 (Avg. price ~$63)\n   - **Tickets**: [Get Tickets](https://atlanta-ga.events/event/gimme-gimme-disco-atlanta-tuesday-december-31-2024-900-pm/?pid=1469)\n\n   ![Gimme Gimme Disco](https://atlanta-ga.events/wp-content/uploads/2023/03/Gimme-Gimme-Disco.jpg)\n\n7. **The Players Ball NYE Experience: Thrice Groove**\n   - **Time**: 9:00 PM\n   - **Venue**: Buckhead Theatre, 3110 Roswell Road, Atlanta, GA\n   - **Prices**: From $44 (Avg. price ~$51)\n   - **Tickets**: [Get Tickets](https://atlanta-ga.events/event/the-players-ball-nye-experience-thrice-groove-atlanta-tuesday-december-31-2024-900-pm/?pid=7935)\n\n8. **Big Freedia**\n   - **Time**: 9:30 PM\n   - **Venue**: City Winery, 675 Ponce De Leon Ave, Atlanta, GA\n   - **Prices**: From $224 (Avg. price ~$258)\n   - **Tickets**: [Get Tickets](https://atlanta-ga.events/event/big-freedia-atlanta-tuesday-december-31-2024-930-pm/?pid=2964)\n\n   ![Big Freedia](https://atlanta-ga.events/wp-content/uploads/2023/03/Big-Freedia.jpg)\n\n9. **Daily Bread**\n   - **Time**: 9:30 PM\n   - **Venue**: The Eastern, 800 Old Flat Shoals Road SE, Atlanta, GA\n   - **Prices**: From $111 (Avg. price ~$158)\n   - **Tickets**: [Get Tickets](https://atlanta-ga.events/event/daily-bread-atlanta-tuesday-december-31-2024-930-pm/?pid=4699)\n\n   ![Daily Bread](https://atlanta-ga.events/wp-content/uploads/2023/09/Daily-Bread.jpg)\n\n10. **Hugel - Artist**\n    - **Time**: 10:00 PM\n    - **Venue**: District Atlanta, 269 Armour Dr NE, Atlanta, GA\n    - **Prices**: From $122 (Avg. price ~$189)\n    - **Tickets**: [Get Tickets](https://atlanta-ga.events/event/hugel-artist-atlanta-tuesday-december-31-2024-1000-pm/?pid=2550)\n\nThese events cover a variety of genres including sports, music concerts, and New Year\u0027s Eve celebrations."
```

I called the tool named ExtractContentFromUrl with these arguments:
```
{"assistantName": "WebPageAgentAnt", "instructions": "Find events in Atlanta on December 31, 2024. https://gosouthatlanta.com/things-to-do/events/atlanta-in-december.html"}
```
and got this result:
```
"Here are events taking place in Atlanta on December 31, 2024:\n\n1. **Zoo Year\u2019s Eve**\n   - **Description**: A daytime New Year\u2019s Eve event at Zoo Atlanta.\n   - **More details**: [Zoo Year\u2019s Eve](https://zooatlanta.org/event/zoo-years-eve/)\n\n2. **New Year\u2019s Eve Celebration**\n   - **Description**: A New Year\u2019s Eve event at the Georgia Aquarium featuring a DJ and dancing, food, and after-hours access to the aquatic exhibits.\n   - **More details**: [New Year\u2019s Eve Celebration](https://www.georgiaaquarium.org/events/event/new-years-eve-2024/)"
```

I called the tool named ExtractContentFromUrl with these arguments:
```
{"assistantName": "WebPageAgentAnt", "instructions": "Find events in Atlanta on December 31, 2024. https://www.eventbrite.com/d/ga--atlanta/december-31/"}
```
and got this result:
```
"Here are some events happening in Atlanta on December 31, 2024:\n\n1. **THE PARTY OF THE CENTURY @ PALM BEACH ATLANTA**\n   - Time: 8:00 PM\n   - Location: Palm Beach ATL\n   - Cost: Free\n   - [Event Link](https://www.eventbrite.com/e/the-party-of-the-century-palm-beach-atlanta-tickets-1123772787659?aff=ebdssbdestsearch)\n   - ![Image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F923458383%2F141658221549%2F1%2Foriginal.20241226-195202?crop=focalpoint\u0026fit=crop\u0026h=230\u0026w=460\u0026auto=format%2Ccompress\u0026q=75\u0026sharp=10\u0026fp-x=0.5\u0026fp-y=0.5\u0026s=fbf8a51e151d9cdc6e0d9af1768e0a07)\n\n2. **NYE PEACHTREE BALL 2025 PRIME ON PEACHTREE**\n   - Time: 9:00 PM\n   - Location: Prime on Peachtree\n   - Cost: From $0.00 (2 for 1 deal available)\n   - [Event Link](https://www.eventbrite.com/e/nye-peachtree-ball-2025-prime-on-peachtree-tues-dec-31st-formal-attire-tickets-1112041669569?aff=ebdssbdestsearch)\n   - ![Image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F922877123%2F154458477488%2F1%2Foriginal.20241224-041059?crop=focalpoint\u0026fit=crop\u0026h=230\u0026w=460\u0026auto=format%2Ccompress\u0026q=75\u0026sharp=10\u0026fp-x=0.820075757576\u0026fp-y=0.193579766537\u0026s=599c9b76ed9894a3d0dd439d14c63c37)\n\n3. **LASTCALL NYE 25**\n   - Time: 7:00 PM\n   - Location: Wicked Wolf Atlanta\n   - Cost: From $0.00\n   - [Event Link](https://www.eventbrite.com/e/lastcall-nye-25-tickets-1123081590269?aff=ebdssbdestsearch)\n   - ![Image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F922603583%2F202161825710%2F1%2Foriginal.20241223-164324?crop=focalpoint\u0026fit=crop\u0026h=230\u0026w=460\u0026auto=format%2Ccompress\u0026q=75\u0026sharp=10\u0026fp-x=0.00507575757576\u0026fp-y=0.00727626459144\u0026s=310e6c284f03f5ac5daf14424a95ed28)\n\n4. **HUGEL NEW YEARS EVE | Make The Girls Dance 2024**\n   - Time: 9:00 PM\n   - Location: District Atlanta\n   - [Event Link](https://www.eventbrite.com/e/hugel-new-years-eve-make-the-girls-dance-2024-district-atlanta-tickets-1013146738167?aff=ebdssbdestsearch)\n   - ![Image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F847414849%2F78855004999%2F1%2Foriginal.20240911-104757?crop=focalpoint\u0026fit=crop\u0026h=200\u0026w=512\u0026auto=format%2Ccompress\u0026q=75\u0026sharp=10\u0026fp-x=0.5\u0026fp-y=0.5\u0026s=03c3b3695114e2467d6971c47e3202fb)\n\n5. **NYE CELEBRATION @ DAIQUIRIVILLE**\n   - Time: 9:00 PM\n   - Location: Underground Atlanta\n   - [Event Link](https://www.eventbrite.com/e/nye-celebration-daiquiriville-tickets-1014766452777?aff=ebdssbdestsearch)\n   - ![Image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F917903283%2F244640390684%2F1%2Foriginal.20241216-013015?crop=focalpoint\u0026fit=crop\u0026h=200\u0026w=400\u0026auto=format%2Ccompress\u0026q=75\u0026sharp=10\u0026fp-x=0.0051832460733\u0026fp-y=0.000700636942675\u0026s=5d586d82a4ec084d81dfbb6c37da1794)\n\nThese are just a few of the events happening on December 31, 2024, in Atlanta. You can find more details and additional events on the [Eventbrite website](https://www.eventbrite.com/d/ga--atlanta/december-31/)."
```

I called the tool named ExtractContentFromUrl with these arguments:
```
{"assistantName": "WebPageAgentAnt", "instructions": "Find events in Atlanta on December 31, 2024. https://discoveratlanta.com/events/seasonal/christmas/"}
```
and got this result:
```
"Here are events happening in Atlanta on December 31, 2024:\n\n1. **Holiday Bricktacular at LEGO Discovery Center**\n   - Celebrate the most wonderful time of the year at LEGO Discovery Center Atlanta.\n   - [More Information](https://discoveratlanta.com/event/detail/holiday-bricktacular-at-lego-discovery-center/)\n\n2. **Westside Sleigh Ride at Westside Motor Lounge**\n   - The holiday season just got a whole lot merrier as Westside Sleigh Ride returns.\n   - [More Information](https://discoveratlanta.com/event/detail/westside-sleigh-ride-at-westside-motor-lounge/)\n\n3. **Candlelight Concerts 2024**\n   - Enjoy beautiful candlelight concerts on December 30-31, 2024.\n   - [More Information](https://discoveratlanta.com/event/detail/candlelight-concerts-2024/)\n\n4. **Free Self-guided Street Art Walking Tours**\n   - Explore the vibrant street art of Atlanta on a self-guided tour available on December 30-31, 2024.\n   - [More Information](https://discoveratlanta.com/event/detail/free-self-guided-street-art-walking-tours/)\n\nThese events offer a variety of experiences to enjoy the holiday season in Atlanta."
```

I called the tool named ExtractContentFromUrl with these arguments:
```
{"assistantName": "WebPageAgentAnt", "instructions": "Find events in Atlanta on December 31, 2024. https://discoveratlanta.com/events/main/"}
```
and got this result:
```
"Here are some events happening in Atlanta on December 31, 2024:\n\n1. **Deck the Hall at the College Football Hall of Fame**\n   - Dates: 12/31/24 \u2013 1/1/25\n   - [More Info](https://discoveratlanta.com/event/detail/deck-the-hall-at-the-college-football-hall-of-fame/)\n   ![Deck the Hall at the College Football Hall of Fame](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734546567/EventEntry/qvbnx6yvfyrhtxxehurq.jpg)\n\n2. **Noon Year\u2019s Eve Family Celebration**\n   - Date: 12/31/24\n   - [More Info](https://discoveratlanta.com/event/detail/noon-year-s-eve-family-celebration/)\n   ![Noon Year\u2019s Eve Family Celebration](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734622597/EventEntry/bp9l8tdm5mmozsq49axy.jpg)\n\n3. **Midnight at Chateau**\n   - Dates: 12/31/24 \u2013 1/1/25\n   - [More Info](https://discoveratlanta.com/event/detail/midnight-at-chateau/)\n   ![Midnight at Chateau](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734542272/EventEntry/ee1l9tcromkg7tdhohey.jpg)\n\n4. **NYE @ May Peel \u2013 A Champagne Story**\n   - Date: 12/31/24\n   - [More Info](https://discoveratlanta.com/event/detail/nye-may-peel-a-champagne-story/)\n   ![NYE @ May Peel \u2013 A Champagne Story](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734536118/EventEntry/tartv5l8esay4enxh1bh.jpg)\n\n5. **New Year\u2019s Eve Bash at The Whitley**\n   - Date: 12/31/24\n   - [More Info](https://discoveratlanta.com/event/detail/new-year-s-eve-bash-at-the-whitley/)\n   ![New Year\u2019s Eve Bash at The Whitley](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1731372920/EventEntry/mtombkqoeqfs3zj02teb.jpg)\n\n6. **New Year\u2019s Eve Black-Tie Dinner at The Waldorf**\n   - Date: 12/31/24\n   - [More Info](https://discoveratlanta.com/event/detail/new-year-s-eve-black-tie-dinner-at-the-waldorf/)\n   ![New Year\u2019s Eve Black-Tie Dinner at The Waldorf](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734621279/EventEntry/dswxoa43zrqdv1ifjqeq.jpg)\n\n7. **New Year\u2019s Eve Black-Tie Gala at The Waldorf**\n   - Dates: 12/31/24 \u2013 1/1/25\n   - [More Info](https://discoveratlanta.com/event/detail/new-year-s-eve-black-tie-gala-at-the-waldorf/)\n   ![New Year\u2019s Eve Black-Tie Gala at The Waldorf](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734633963/EventEntry/fwhenoca5zs8whpy2vmq.jpg)\n\n8. **Noon Year\u2019s Eve at LEGO Discovery Center Atlanta**\n   - Date: 12/31/24\n   - [More Info](https://discoveratlanta.com/event/detail/noon-year-s-eve-at-lego-discovery-center-atlanta/)\n   ![Noon Year\u2019s Eve at LEGO Discovery Center Atlanta](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734577971/EventEntry/mbgc1yk4glqiyq3zxnxs.jpg)\n\n9. **Toast to 2025: Amore e Amore Rings in the New Year**\n   - Date: 12/31/24\n   - [More Info](https://discoveratlanta.com/event/detail/toast-to-2025-amore-e-amore-rings-in-the-new-year/)\n   ![Toast to 2025: Amore e Amore Rings in the New Year](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734541369/EventEntry/aoouncrlkwrd5rdbyt3z.jpg)\n\n10. **Atlanta\u2019s New Year\u2019s Eve Peach Drop Festival**\n    - Dates: 12/31/24 \u2013 1/1/25\n    - [More Info](https://discoveratlanta.com/event/detail/atlanta-s-new-year-s-eve-peach-drop-festival/)\n    ![Atlanta\u2019s New Year\u2019s Eve Peach Drop Festival](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1669996517/EventEntry/wvln4rofrjlcjpilbub1.jpg)\n\nThese events provide a variety of entertainment options ranging from family-friendly activities to elegant gala dinners."
```

Here are some exciting events happening in Atlanta on December 31, 2024:

1. **Georgia Tech Yellow Jackets vs. Notre Dame Fighting Irish**
   - **Time**: 2:30 PM
   - **Venue**: McCamish Pavilion, 965 Fowler St., Atlanta, GA
   - **Tickets**: [Get Tickets](https://atlanta-ga.events/event/georgia-tech-yellow-jackets-vs-notre-dame-fighting-irish-atlanta-tuesday-december-31-2024-230-pm/?pid=1282)
   ![Georgia Tech Yellow Jackets Basketball](https://atlanta-ga.events/wp-content/uploads/2023/08/Georgia-Tech-Yellow-Jackets-Basketball.jpg)

2. **Provenance NYE: Seven Lions, Knife Party & Grabbitz**
   - **Time**: 7:00 PM
   - **Venue**: Atlanta Coliseum, 2075 Market St, Duluth, GA
   - **Tickets**: [Get Tickets](https://atlanta-ga.events/event/provenance-nye-seven-lions-knife-party-and-grabbitz-duluth-tuesday-december-31-2024-700-pm/?pid=8320)
   ![Provenance NYE](https://atlanta-ga.events/wp-content/uploads/2023/03/Seven-Lions.jpg)

3. **Zoo Year’s Eve**
   - **Description**: A daytime New Year’s Eve event at Zoo Atlanta.
   - **More details**: [Zoo Year’s Eve](https://zooatlanta.org/event/zoo-years-eve/)

4. **New Year’s Eve Celebration at the Georgia Aquarium**
   - **Description**: Featuring a DJ and dancing, food, and after-hours access to the aquatic exhibits.
   - **More details**: [New Year’s Eve Celebration](https://www.georgiaaquarium.org/events/event/new-years-eve-2024/)

5. **HUGEL NEW YEARS EVE | Make The Girls Dance 2024**
   - **Time**: 9:00 PM
   - **Venue**: District Atlanta
   - **Tickets**: [Event Link](https://www.eventbrite.com/e/hugel-new-years-eve-make-the-girls-dance-2024-district-atlanta-tickets-1013146738167?aff=ebdssbdestsearch)
   ![HUGEL NEW YEARS EVE](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F847414849%2F78855004999%2F1%2Foriginal.20240911-104757?crop=focalpoint&fit=crop&h=200&w=512&auto=format%2Ccompress&q=75&sharp=10&fp-x=0.5&fp-y=0.5&s=03c3b3695114e2467d6971c47e3202fb)

6. **Atlanta’s New Year’s Eve Peach Drop Festival**
   - **Dates**: 12/31/24 – 1/1/25
   - **More Info**: [Peach Drop Festival](https://discoveratlanta.com/event/detail/atlanta-s-new-year-s-eve-peach-drop-festival/)
   ![Peach Drop Festival](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1669996517/EventEntry/wvln4rofrjlcjpilbub1.jpg)

These events cover a variety of interests, including sports, music concerts, family-friendly activities, and New Year's Eve celebrations. Enjoy your time in Atlanta!


In [5]:
output.LastMessage.DisplayAs("text/markdown");

Here are some exciting events happening in Atlanta on December 31, 2024:

1. **Georgia Tech Yellow Jackets vs. Notre Dame Fighting Irish**
   - **Time**: 2:30 PM
   - **Venue**: McCamish Pavilion, 965 Fowler St., Atlanta, GA
   - **Tickets**: [Get Tickets](https://atlanta-ga.events/event/georgia-tech-yellow-jackets-vs-notre-dame-fighting-irish-atlanta-tuesday-december-31-2024-230-pm/?pid=1282)
   ![Georgia Tech Yellow Jackets Basketball](https://atlanta-ga.events/wp-content/uploads/2023/08/Georgia-Tech-Yellow-Jackets-Basketball.jpg)

2. **Provenance NYE: Seven Lions, Knife Party & Grabbitz**
   - **Time**: 7:00 PM
   - **Venue**: Atlanta Coliseum, 2075 Market St, Duluth, GA
   - **Tickets**: [Get Tickets](https://atlanta-ga.events/event/provenance-nye-seven-lions-knife-party-and-grabbitz-duluth-tuesday-december-31-2024-700-pm/?pid=8320)
   ![Provenance NYE](https://atlanta-ga.events/wp-content/uploads/2023/03/Seven-Lions.jpg)

3. **Zoo Year’s Eve**
   - **Description**: A daytime New Year’s Eve event at Zoo Atlanta.
   - **More details**: [Zoo Year’s Eve](https://zooatlanta.org/event/zoo-years-eve/)

4. **New Year’s Eve Celebration at the Georgia Aquarium**
   - **Description**: Featuring a DJ and dancing, food, and after-hours access to the aquatic exhibits.
   - **More details**: [New Year’s Eve Celebration](https://www.georgiaaquarium.org/events/event/new-years-eve-2024/)

5. **HUGEL NEW YEARS EVE | Make The Girls Dance 2024**
   - **Time**: 9:00 PM
   - **Venue**: District Atlanta
   - **Tickets**: [Event Link](https://www.eventbrite.com/e/hugel-new-years-eve-make-the-girls-dance-2024-district-atlanta-tickets-1013146738167?aff=ebdssbdestsearch)
   ![HUGEL NEW YEARS EVE](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F847414849%2F78855004999%2F1%2Foriginal.20240911-104757?crop=focalpoint&fit=crop&h=200&w=512&auto=format%2Ccompress&q=75&sharp=10&fp-x=0.5&fp-y=0.5&s=03c3b3695114e2467d6971c47e3202fb)

6. **Atlanta’s New Year’s Eve Peach Drop Festival**
   - **Dates**: 12/31/24 – 1/1/25
   - **More Info**: [Peach Drop Festival](https://discoveratlanta.com/event/detail/atlanta-s-new-year-s-eve-peach-drop-festival/)
   ![Peach Drop Festival](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1669996517/EventEntry/wvln4rofrjlcjpilbub1.jpg)

These events cover a variety of interests, including sports, music concerts, family-friendly activities, and New Year's Eve celebrations. Enjoy your time in Atlanta!

In [6]:
assistantRunOptions = new AssistantRunOptions() {
    AssistantName = "WebSearchAgentAnt",
    Instructions = "Use python to get todays date and then use it to find things to do in Atlanta for each of the next four weekends with pictures and correct links from ExtractContentFromUrl.\n\nWhen choosing links for crawl to evaluate with ExtractContentFromUrl choose a variety of hosts for urls from which you extract content. Limit your use of ExtractContentFromUrl to five calls or less. Do not make the error of including the url as a parameter, it must be inside 'instructions' as indicated by the tool definition. Be careful to not include events in the past in your final answer. You should perform four searches at once.\nDon't forget to include valid links!",
};
var output2 = await AntRunnerLib.AssistantRunner.RunThread(assistantRunOptions, config);
output2.Dialog.DisplayAs("text/markdown");


User: Use python to get todays date and then use it to find things to do in Atlanta for each of the next four weekends with pictures and correct links from ExtractContentFromUrl.

When choosing links for crawl to evaluate with ExtractContentFromUrl choose a variety of hosts for urls from which you extract content. Limit your use of ExtractContentFromUrl to five calls or less. Do not make the error of including the url as a parameter, it must be inside 'instructions' as indicated by the tool definition. Be careful to not include events in the past in your final answer. You should perform four searches at once.
Don't forget to include valid links!

Assistant: I ran this code:
```
from datetime import datetime, timedelta

# Get today's date
today = datetime.now()

# Calculate the dates for the next four weekends
# A weekend is considered Saturday and Sunday
weekends = []
for i in range(1, 5):
    saturday = today + timedelta(days=(5 - today.weekday()) + (i - 1) * 7)  # Get the next Saturday
    sunday = saturday + timedelta(days=1)  # Get the next Sunday
    weekends.append((saturday, sunday))

weekends
```

I called the tool named crawl with these arguments:
```
{"q": "things to do in Atlanta January 4-5 2025", "textFormat": "Raw"}
```
and got this result:
```
{"queryContext":{"originalQuery":"things to do in Atlanta January 4-5 2025"},"webPages":{"value":[{"name":"Atlanta Events January 2025: Concerts, Shows, Sports...","url":"https://atlanta-ga.events/january/","displayUrl":"https://atlanta-ga.events/january","snippet":"Explore our comprehensive calendar for all the current and upcoming events in Atlanta, GA, for January 2025. This guide is your gateway to discovering the best things to do in Atlanta, from Concerts and Theater Shows to Sports events and music festivals. You\u2019ll find a detailed list of all the most engaging and exciting activities happening throughout the month."},{"name":"Atlanta Events \u2013 January 2025 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-january.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-january.html","snippet":"Events From January 1 To The Weekend Of January 4-5 2025. \u2192 See December events. \u2192 For concerts this week, see concerts in Atlanta in January \u2013 IllumiNights, November 14 2024 - January 19 2025 ... \u2013 Atlanta Boat Show, January 9-12 2025 Annual boating trade show at the Georgia World Congress Center. More details \u2013 Scott Antique Market ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.xm-3dn3hYJZHsVVU9WO5qwHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"8 Things to Do in Atlanta This Weekend: Jan. 3-5, 2025 - Time Out","url":"https://www.timeout.com/atlanta/things-to-do/best-things-to-do-in-atlanta-this-weekend","displayUrl":"https://www.timeout.com/atlanta/things-to-do/best-things-to-do-in-atlanta-this-weekend","snippet":"Jan. 4 at Cobb Energy Centre. Get ready to loop through memory lane at Sonic Symphony, a live concert celebrating over 30 years of iconic Sonic the Hedgehog music. From retro 8-bit classics to ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.wA_rMsgTDiYoT68iqJWOpgHaFj\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Discover 2025 Events \u0026 Activities in Atlanta, GA | Eventbrite","url":"https://www.eventbrite.com/d/ga--atlanta/2025/","displayUrl":"https://www.eventbrite.com/d/ga--atlanta/2025","snippet":"Save this event: From Day One: Atlanta 2025 Share this event: From Day One: Atlanta 2025. From Day One: Atlanta 2025. Tue, Jan 28, 8:30 AM. Georgia Aquarium. ... NuNu\u0027s Disco Birthday Brunch Jan 5th 2025 Share this event: NuNu\u0027s Disco Birthday Brunch Jan 5th 2025. NuNu\u0027s Disco Birthday Brunch Jan 5th 2025. Sun, Jan 5, 12:00 PM."},{"name":"Events \u0026 Things To Do in Atlanta in 2025 - AllEvents.in","url":"https://allevents.in/atlanta/january","displayUrl":"https://allevents.in/atlanta/january","snippet":"Variety Playhouse, Atlanta: 18 Jan 2025: Fulton Lee at The Loft at Center Stage: The Loft at Center Stage, Atlanta: 23 Jan 2025: Kathleen Edwards at City Winery - Atlanta: City Winery - Atlanta: 27 Jan 2025: MJ Lenderman at Variety Playhouse: Variety Playhouse, Atlanta: 28 Jan 2025: Life Love and Beyond Podcast at City Winery - Atlanta: City ..."},{"name":"Atlanta Events Calendar 2024/2025 | Atlanta, GA","url":"https://atlanta-ga.events/","displayUrl":"https://atlanta-ga.events","snippet":"Atlanta Events 2024/2025. Discover the calendar of all current and upcoming events in Atlanta, GA. This calendar is your ultimate guide to the best things to do in Atlanta, GA: ... Jan 4, 2025. Category. Search Events Calendar. Jan Feb Mar Apr May Jun Jul Aug Sep ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.ytkv04WKv8RTyYo4_DsP4gHaHa\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"20 Absolutely Fantastic Things To Do In Atlanta This January","url":"https://secretatlanta.co/things-to-do-january/","displayUrl":"https://secretatlanta.co/things-to-do-january","snippet":"Oh, and it\u2019s a wonderful time to try out some brand new things too! If you\u2019re looking for some fun things to do around Atlanta this January, don\u2019t worry, we\u2019ve got ya covered! Keep scrolling to see all the best things to do this January in Atlanta! 1. Center yourself at Oasis at the High Museum of Art High Museum of Art","thumbnailUrl":"https://www.bing.com/th?id=OIP.5lOsdA8tsTOgx5qTd465YAAAAA\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta Events | Explore Georgia","url":"https://exploregeorgia.org/atlanta/events","displayUrl":"https://exploregeorgia.org/atlanta/events","snippet":"Browse the ideas and Atlanta events below to help plan your trip. Find Events. Arts \u0026 Culture (21) Culinary, Food, Wine \u0026 Beer (12) Entertainment (67) Fairs \u0026 Festivals (21) ... January 4, 2025 - May 24, 2025 Painting Zoo Painters. Zoo Atlanta. View all dates. Atlanta. Sunday, January 5. January 5, 2025 - April 27, 2025","thumbnailUrl":"https://www.bing.com/th?id=OIP.qcVhz7_S17NbT6s7cPa8nQHaEi\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Cool Things To Do in January in Atlanta","url":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-january-in-atlanta/","displayUrl":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-january-in-atlanta","snippet":"Cool Things To Do in February in Atlanta; Cool Things To Do in December in Atlanta; Cool Things to Do in March in Atlanta; Family-Friendly Things To Do. Grab the whole family to explore the city, all while trying to keep warm. Delta Flight Museum. 2025 marks the 100th anniversary of Delta Air Lines.","thumbnailUrl":"https://www.bing.com/th?id=OIP.aU9EcatRnz4MDkIKyYbkQgHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta in January | Concerts, Festivals, Tickets \u0026 Tour Dates - Songkick","url":"https://www.songkick.com/metro-areas/4120-us-atlanta/january-2025","displayUrl":"https://www.songkick.com/metro-areas/4120-us-atlanta/january-2025","snippet":"Concerts in Atlanta in January. Find tickets to all live music, concerts, tour dates and festivals in and around Atlanta. Currently there are 64 upcoming events. Filter by artist. ... Thursday 02 January 2025; Joe Alterman. Eddie\u0027s Attic, Decatur, GA, US . Friday 03 January 2025; Raheem DeVaughn."}]}}
```

I called the tool named crawl with these arguments:
```
{"q": "things to do in Atlanta January 11-12 2025", "textFormat": "Raw"}
```
and got this result:
```
{"queryContext":{"originalQuery":"things to do in Atlanta January 11-12 2025"},"webPages":{"value":[{"name":"Atlanta Events \u2013 January 2025 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-january.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-january.html","snippet":"Things to do in Atlanta, GA in January: calendar of upcoming festivals, Martin Luther King Day events, comedy, musicals, races, conventions, and other events. Go South! ... January 11-12 2025 Annual travel and tourism trade show at the Georgia International Convention Center. More details \u2013 Love\u2019s Labour\u2019s Lost, January 11-26 2025 ..."},{"name":"Atlanta Events January 2025: Concerts, Shows, Sports...","url":"https://atlanta-ga.events/january/","displayUrl":"https://atlanta-ga.events/january","snippet":"Explore our comprehensive calendar for all the current and upcoming events in Atlanta, GA, for January 2025. This guide is your gateway to discovering the best things to do in Atlanta, from Concerts and Theater Shows to Sports events and music festivals. You\u2019ll find a detailed list of all the most engaging and exciting activities happening throughout the month.","thumbnailUrl":"https://www.bing.com/th?id=OIP.ytkv04WKv8RTyYo4_DsP4gHaHa\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"January 11 events in Atlanta, GA - Eventbrite","url":"https://www.eventbrite.com/d/ga--atlanta/january-11/","displayUrl":"https://www.eventbrite.com/d/ga--atlanta/january-11","snippet":"Save this event: 2025 Atlanta Travel \u0026 Adventure Show Share this event: ... Sat, Jan 11, 12:00 PM. The Golden Door Event Hall. Share this event: Powerful Dialogue: ... Sat, Jan 11, 4:00 PM. Atlanta\u0027s Original Uptown Comedy Corner (225) Share this event: ..."},{"name":"Atlanta January 2025: Events, Concerts, Clubs \u0026 Things to Do - Experism","url":"https://www.experism.com/atlanta/january","displayUrl":"https://www.experism.com/atlanta/january","snippet":"Things to do in Atlanta January Atlanta Nightclub Events in January ... Jan 9-12, 2025 Georgia World Congress Center Hall C View on Map. The Atlanta Boat Show is an annual event that showcases the latest and greatest in boating technology and accessories. ... Jan 11, 2025 Atlanta View on Map. Don\u0027t miss two-time Emmy Award-winning comedian ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.WKXWyOtbF2Q-oMmg6Y-HiQHaE8\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Discover 2025 Events \u0026 Activities in Atlanta, GA | Eventbrite","url":"https://www.eventbrite.com/d/ga--atlanta/2025/","displayUrl":"https://www.eventbrite.com/d/ga--atlanta/2025","snippet":"Vision Board 2025. Sun, Jan 12, 12:00 PM. 3107 Peachtree Rd NE a1. Share this event: ... 2025 Atlanta Travel \u0026 Adventure Show Share this event: ... Atlanta Travel \u0026 Adventure Show Save this event: 2025 Atlanta Travel \u0026 Adventure Show. ELEVATE : IGNITE CONNECTIONS FOR 2025. Sat, Jan 11, 5:00 PM. The Ritz-Carlton, Atlanta. Save this event ..."},{"name":"20 Absolutely Fantastic Things To Do In Atlanta This January","url":"https://secretatlanta.co/things-to-do-january/","displayUrl":"https://secretatlanta.co/things-to-do-january","snippet":"If you\u2019re looking for some fun things to do around Atlanta this January, don\u2019t worry, we\u2019ve got ya covered! ... 12. Check out one of Atlanta\u2019s lesser-known museums ... Where: Woodruff Park | 91 Peachtree Street, Atlanta, GA. When: Until January 12th, 2025. If you\u2019re searching for a dazzling display to check out this January, don\u2019t ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.5lOsdA8tsTOgx5qTd465YAAAAA\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Cool Things To Do in January in Atlanta","url":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-january-in-atlanta/","displayUrl":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-january-in-atlanta","snippet":"Find cool things to do in Atlanta during the month of January, from annual festivals to seasonal activities, theatrical performances, and more! ... Coucheron \u002B The Four Seasons, Jan. 10-12. Joe Gatto: LET\u2019S GET INTO IT, Jan. 11. Rachmaninoff \u002B Stravinsky, Jan. 16-18. Music for the Very Young \u2013 The Lion and the Mouse: A Musical Tale for ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.aU9EcatRnz4MDkIKyYbkQgHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"20 free things to do in metro Atlanta in January","url":"https://www.accessatlanta.com/atlanta-things-to-do/20-free-things-to-do-in-metro-atlanta-in-january/Q3EOZY6OHBE2HLT4QN5ZIBLBQU/","displayUrl":"https://www.accessatlanta.com/atlanta-things-to-do/20-free-things-to-do-in-metro...","snippet":"Here are 20 of the best things to do in Atlanta in January that are totally free. ... Date: Saturday, Jan. 11; Time: 9-10:30 a.m. Bearing Witness: Unforgettable Stories from the Holocaust - The ...","thumbnailUrl":"https://www.bing.com/th?id=OIF.D7TspzC8FCu49gY46WNjCQ\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Fun, Free Activities for Kids in January 2025: 15 Free Things To Do in ...","url":"https://www.msn.com/en-us/sports/other/fun-free-activities-for-kids-in-january-2025-15-free-things-to-do-in-atlanta-with-kids/ar-AA1mOEey","displayUrl":"https://www.msn.com/en-us/sports/other/fun-free-activities-for-kids-in-january-2025-15...","snippet":"Free Things to Do with Kids in Atlanta January 2025. Chick-fil-A Peach Bowl Fan Fest Downtown. Wednesday, January 1, 2025. ... Sunday, January 12, 2025. On the second Sunday of each month ..."},{"name":"Atlanta in January | Concerts, Festivals, Tickets \u0026 Tour Dates - Songkick","url":"https://www.songkick.com/metro-areas/4120-us-atlanta/january-2025","displayUrl":"https://www.songkick.com/metro-areas/4120-us-atlanta/january-2025","snippet":"Concerts in Atlanta in January Find tickets to all live music, concerts, tour dates and festivals in and around Atlanta. Currently there are 64 upcoming events."}]},"images":{"value":[{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.5jERbhhnl8LiECUQySoRYAHaG1\u0026pid=Api","contentUrl":"https://uphomes.com/storage/blogposts/February2023/medium/RKuDJcqErH176HD3LyWV.png","hostPageUrl":"https://uphomes.com/blog/things-to-do-atlanta-ga/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.W-gOzzcskaYfPXC3whhXOgHaLH\u0026pid=Api","contentUrl":"https://www.busytourist.com/wp-content/uploads/2020/07/best-things-to-do-in-Atlanta-GA-1.jpg","hostPageUrl":"https://www.busytourist.com/things-to-do-in-atlanta-ga/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.ENao3rD4rUbnHGA6QJHXYgAAAA\u0026pid=Api","contentUrl":"https://res.cloudinary.com/hz3gmuqw6/image/upload/c_fill,q_auto,w_750/f_auto/things-to-do-in-atlanta-phpWlZ13b","hostPageUrl":"https://www.cozymeal.com/magazine/things-to-do-in-atlanta"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.BawFpewdQrecAtgiS9AiOAHaHa\u0026pid=Api","contentUrl":"https://unexpectedatlanta.com/wp-content/uploads/2023/04/The-Best-Things-to-Do-in-Midtown-Atlanta-500x500.png","hostPageUrl":"https://unexpectedatlanta.com/best-things-to-do-midtown-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.Fe-J-AR_X-7c4ZeUHVaHwQAAAA\u0026pid=Api","contentUrl":"https://www.acaponeconnection.com/wp-content/uploads/2022/05/All-the-Fun-Things-to-Do-in-Atlanta-Georgia-1.jpg","hostPageUrl":"https://www.acaponeconnection.com/all-the-fun-things-to-do-in-atlanta-georgia/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.uXSLRLfe7U8MxUYx6YGGVAHaE7\u0026pid=Api","contentUrl":"https://quartzmountain.org/images/resources/things-to-do-atlanta-january_20230726190854.webp","hostPageUrl":"https://quartzmountain.org/article/things-to-do-atlanta-january"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.fqwDUYiMaHe0id6KtJYyfwAAAA\u0026pid=Api","contentUrl":"https://unexpectedatlanta.com/wp-content/uploads/2022/12/Visiting-Atlanta-in-January-Ultimate-Guide-1.png","hostPageUrl":"https://unexpectedatlanta.com/atlanta-in-january/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.vjZ_V7Yjzfqyphfsj_SBjgHaEK\u0026pid=Api","contentUrl":"https://daytripqueen.com/wp-content/uploads/2021/07/Free-things-to-do-in-Atlanta.jpg","hostPageUrl":"https://daytripqueen.com/free-things-to-do-in-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.FFnn7XEC9iNqbbTqDM01LQHaD3\u0026pid=Api","contentUrl":"https://www.busytourist.com/wp-content/uploads/2020/07/things-to-do-in-Atlanta-803x420.jpg","hostPageUrl":"https://www.busytourist.com/things-to-do-in-atlanta-ga/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.QzxRYsvq-SWPZ-nndzHXNAHaLH\u0026pid=Api","contentUrl":"https://365atlantatraveler.com/wp-content/uploads/2019/03/things-to-do-atlanta.jpg","hostPageUrl":"https://telegra.ph/Asian-senior-citizen-activities-atlanta-03-12"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.yRC0krj1Jbi8qtYhSLLimgHaE8\u0026pid=Api","contentUrl":"https://lp-cms-production.imgix.net/2021-05/GettyRF_486068945.jpg?auto=compress\u0026fit=crop\u0026fm=auto\u0026sharp=10\u0026vib=20\u0026w=1200\u0026h=800","hostPageUrl":"https://www.lonelyplanet.com/articles/top-things-to-do-in-atlanta"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.VX9GFXDDPaUhlk5Ajz2jFAHaLH\u0026pid=Api","contentUrl":"https://www.busytourist.com/wp-content/uploads/2020/07/best-things-to-do-in-Atlanta-GA-1-2.jpg","hostPageUrl":"https://www.busytourist.com/things-to-do-in-atlanta-ga/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.MdoK5WF-Bvvg72JvIonreQHaEG\u0026pid=Api","contentUrl":"https://stonehurstplace.com/wp-content/uploads/2020/01/Things-to-do-in-January-in-Atlanta-1100x609-1.png","hostPageUrl":"https://stonehurstplace.com/our-diary/attractions/things-to-do-in-atlanta-in-january/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.JIWcXtQtpq7A0DXTjwLQiwHaLG\u0026pid=Api","contentUrl":"https://i.pinimg.com/736x/fc/4a/d2/fc4ad281802906d1cfde53017a3e4dcf.jpg","hostPageUrl":"https://www.pinterest.com/pin/744079169673803460/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.v6IPB_NTvbmy-pxyZSCA0QHaE7\u0026pid=Api","contentUrl":"http://rwethereyetmom.com/wp-content/uploads/2015/08/6-Things-to-do-in-Atlanta-1024x681.jpg","hostPageUrl":"https://rwethereyetmom.com/6-things-to-do-in-atlanta.html"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.8B9LmuRXqgsRTEY6n_RNlAHaHa\u0026pid=Api","contentUrl":"https://unexpectedatlanta.com/wp-content/uploads/2023/06/The-Best-Things-to-Do-in-Atlanta.png","hostPageUrl":"https://unexpectedatlanta.com/atlanta-event-calendar/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.Twqjja_QS8r-DEoNYr3kzAHaEK\u0026pid=Api","contentUrl":"https://i.ytimg.com/vi/Vl6GPUuj1mc/maxresdefault.jpg","hostPageUrl":"https://www.youtube.com/watch?v=Vl6GPUuj1mc"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.IqznTH1ocAOkn-XVKO4tyQHaL5\u0026pid=Api","contentUrl":"https://i.pinimg.com/originals/f5/05/8e/f5058e44214459ab6506002aee8aa99f.png","hostPageUrl":"https://www.pinterest.jp/pin/101-best-things-to-do-in-atlanta-georgia-the-ultimate-attractions-guide--34480753388071714/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.jU8QTHjizge3CjySqF-SrwHaE7\u0026pid=Api","contentUrl":"https://a.cdn-hotels.com/gdcs/production140/d580/4881c5aa-5a32-4a10-8fed-9617f9926700.jpg","hostPageUrl":"https://uk.hotels.com/go/usa/things-to-do-atlanta"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.OD8mHt8EuAz_-Rm6pRiOFwHaEK\u0026pid=Api","contentUrl":"https://i.ytimg.com/vi/2ufhW2kEYro/maxresdefault.jpg","hostPageUrl":"https://www.youtube.com/watch?v=2ufhW2kEYro"}]}}
```

I called the tool named crawl with these arguments:
```
{"q": "things to do in Atlanta January 18-19 2025", "textFormat": "Raw"}
```
and got this result:
```
{"queryContext":{"originalQuery":"things to do in Atlanta January 18-19 2025"},"webPages":{"value":[{"name":"Atlanta Events January 2025: Concerts, Shows, Sports...","url":"https://atlanta-ga.events/january/","displayUrl":"https://atlanta-ga.events/january","snippet":"Explore our comprehensive calendar for all the current and upcoming events in Atlanta, GA, for January 2025. This guide is your gateway to discovering the best things to do in Atlanta, from Concerts and Theater Shows to Sports events and music festivals. You\u2019ll find a detailed list of all the most engaging and exciting activities happening throughout the month.","thumbnailUrl":"https://www.bing.com/th?id=OIP.ytkv04WKv8RTyYo4_DsP4gHaHa\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta Events \u2013 January 2025 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-january.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-january.html","snippet":"Things to do in Atlanta, GA in January: calendar of upcoming festivals, Martin Luther King Day events, comedy, musicals, races, conventions, and other events. ... \u2013 IllumiNights, November 14 2024 - January 19 2025 Annual after-hours walk-through holiday lights show at Zoo Atlanta, with a mile of illuminated animal lanterns. ... January 18-20 ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.xm-3dn3hYJZHsVVU9WO5qwHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Events \u0026 Things To Do in Atlanta in 2025 - AllEvents.in","url":"https://allevents.in/atlanta/january","displayUrl":"https://allevents.in/atlanta/january","snippet":"Find events and things to do in January 2025 in Atlanta. Discover parties, concerts, meets, shows, sports, club, reunion, Performance happening in January 2024 in Atlanta. Open Menu. ... Atlanta: 18 Jan 2025: Electric Avenue at Variety Playhouse: Variety Playhouse, Atlanta: 18 Jan 2025: Fulton Lee at The Loft at Center Stage:"},{"name":"Cool Things To Do in January in Atlanta","url":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-january-in-atlanta/","displayUrl":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-january-in-atlanta","snippet":"Find cool things to do in Atlanta during the month of January, from annual festivals to seasonal activities, theatrical performances, and more! ... Music for the Very Young \u2013 The Lion and the Mouse: A Musical Tale for Little Ears, Jan. 18-19. SWAG Gospel, Jan. 19. Beethoven Project: Symphony No. 1 and Symphony No. 3, Jan. 23-25. IN UNISON ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.aU9EcatRnz4MDkIKyYbkQgHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Fun, Free Activities for Kids in January 2025: 15 Free Things To Do in ...","url":"https://www.msn.com/en-us/sports/other/fun-free-activities-for-kids-in-january-2025-15-free-things-to-do-in-atlanta-with-kids/ar-AA1mOEey","displayUrl":"https://www.msn.com/en-us/sports/other/fun-free-activities-for-kids-in-january-2025-15...","snippet":"Free Things to Do with Kids in Atlanta January 2025. Chick-fil-A Peach Bowl Fan Fest Downtown. ... January 18, 2025. Mini-DIYers head to Lowe\u2019s to attend a workshop hosted by a Lowes employee ..."},{"name":"20 free things to do in metro Atlanta in January","url":"https://www.accessatlanta.com/atlanta-things-to-do/20-free-things-to-do-in-metro-atlanta-in-january/Q3EOZY6OHBE2HLT4QN5ZIBLBQU/","displayUrl":"https://www.accessatlanta.com/atlanta-things-to-do/20-free-things-to-do-in-metro...","snippet":"Here are 20 of the best things to do in Atlanta in January that are ... Kick off 2025 in style with Yoga \u0026 Sound at The ... Website: atlantahistorycenter.com; Date: Saturday, Jan 18; Time: 10 a.m ...","thumbnailUrl":"https://www.bing.com/th?id=OIF.D7TspzC8FCu49gY46WNjCQ\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Explore Atlanta Events for Today, This Week or Weekend","url":"https://discoveratlanta.com/events/main/","displayUrl":"https://discoveratlanta.com/events","snippet":"Explore the Atlanta events calendar for things to do today, this week or this weekend. Find upcoming festivals, seasonal events and more! ... Deftones North American Tour 2025 3/18/25 \u2013 3/19/25. Milo the Magnificent 3/19/25 \u2013 3/30/25. J Balvin Live at State Farm Arena ... January 19. Sunday. CFP Taste of the Championship Atlanta 1/19/25 ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.FkxSV6JbrWatsFMM3_8dIAHaEh\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Top Attractions to Visit this Winter in Atlanta","url":"https://discoveratlanta.com/stories/things-to-do/attractions-to-visit-this-winter-in-atlanta/","displayUrl":"https://discoveratlanta.com/stories/things-to-do/attractions-to-visit-this-winter-in...","snippet":"Children\u2019s Museum of Atlanta offers families an opportunity to come together with a new exhibit from Jan. 25, 2025, through May 4, 2025. \u201CXOXO: An Exhibit About Love \u0026 Forgiveness\u201D is designed to foster conversation and connection and is designed for guests of all ages. Where: 275 Centennial Olympic Park Drive NW","thumbnailUrl":"https://www.bing.com/th?id=OIP.z8MiD8hlIeihw0QjVGJzrgHaE7\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"20 Absolutely Fantastic Things To Do In Atlanta This January","url":"https://secretatlanta.co/things-to-do-january/","displayUrl":"https://secretatlanta.co/things-to-do-january","snippet":"If you\u2019re looking for some fun things to do around Atlanta this January, don\u2019t worry, we\u2019ve got ya covered! ... Woodruff Park | 91 Peachtree Street, Atlanta, GA. When: Until January 12th, 2025. If you\u2019re searching for a dazzling display to check out this January, don\u2019t worry, we\u2019ve got you covered! ... 18. Set sail on an emotional ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.5lOsdA8tsTOgx5qTd465YAAAAA\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"23 Things You Have to Do in Atlanta This Year - Thrillist","url":"https://www.thrillist.com/events/atlanta/events-calendar-in-atlanta-this-year","displayUrl":"https://www.thrillist.com/events/atlanta/events-calendar-in-atlanta-this-year","snippet":"January 17\u201320, 2025 ... February 5\u20137, 2025 SCAD Atlanta SCAD TVfest returns in 2025, bringing the best in television, streaming, and digital media to Atlanta for its 12th year. This premier ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.WUHCPl7_RpepE3pSEdAEMgHaE8\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"}]},"images":{"value":[{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.WoE56TfoQ63T7_SpwvmGKgHaE8\u0026pid=Api","contentUrl":"https://www.experism.com/explore/wp-content/uploads/2023/02/Atlanta-1550x1034.jpg","hostPageUrl":"https://www.experism.com/atlanta/january"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.5jERbhhnl8LiECUQySoRYAHaG1\u0026pid=Api","contentUrl":"https://uphomes.com/storage/blogposts/February2023/medium/RKuDJcqErH176HD3LyWV.png","hostPageUrl":"https://uphomes.com/blog/things-to-do-atlanta-ga/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.-1BVEwbN81Oc9jxAOaX2SQHaE5\u0026pid=Api","contentUrl":"https://southerntrippers.com/wp-content/uploads/2022/08/things-to-do-in-atlanta-at-night-centennial-olympic-park-.jpg","hostPageUrl":"https://nertqherminia.pages.dev/yfskiot-things-to-do-in-atlanta-georgia-in-january-2024-wwtcyrq/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.k7hGzw27Lx3uqX8C1eseEQHaE6\u0026pid=Api","contentUrl":"https://365atlantatraveler.com/wp-content/uploads/2022/05/atlanta-1280x850.jpeg","hostPageUrl":"https://nertqherminia.pages.dev/yfskiot-things-to-do-in-atlanta-georgia-in-january-2024-wwtcyrq/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.z9bHreP8_hqtZ1wfgtW3NQHaE8\u0026pid=Api","contentUrl":"https://lp-cms-production.imgix.net/2021-05/GettyRF_486068945.jpg?auto=format\u0026q=40\u0026w=870\u0026dpr=5","hostPageUrl":"https://www.lonelyplanet.com/articles/best-parks-atlanta"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.fqwDUYiMaHe0id6KtJYyfwAAAA\u0026pid=Api","contentUrl":"https://unexpectedatlanta.com/wp-content/uploads/2022/12/Visiting-Atlanta-in-January-Ultimate-Guide-1.png","hostPageUrl":"https://unexpectedatlanta.com/atlanta-in-january/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.uXSLRLfe7U8MxUYx6YGGVAHaE7\u0026pid=Api","contentUrl":"https://quartzmountain.org/images/resources/things-to-do-atlanta-january_20230726190854.webp","hostPageUrl":"https://quartzmountain.org/article/things-to-do-atlanta-january"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.FFnn7XEC9iNqbbTqDM01LQHaD3\u0026pid=Api","contentUrl":"https://www.busytourist.com/wp-content/uploads/2020/07/things-to-do-in-Atlanta-803x420.jpg","hostPageUrl":"https://porthidden.weebly.com/blog/free-things-to-do-in-atlanta-georgia"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.ENao3rD4rUbnHGA6QJHXYgAAAA\u0026pid=Api","contentUrl":"https://res.cloudinary.com/hz3gmuqw6/image/upload/c_fill,q_auto,w_750/f_auto/things-to-do-in-atlanta-phpWlZ13b","hostPageUrl":"https://www.cozymeal.com/magazine/things-to-do-in-atlanta"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.AsEhWz77ez1CIx9RVZsvFAHaEh\u0026pid=Api","contentUrl":"https://res.cloudinary.com/atlanta/images/v1636654462/newAtlanta.com/Atlanta_OneMusicFest_COP-1/Atlanta_OneMusicFest_COP-1.jpg?_i=AA","hostPageUrl":"https://karylbridgette.pages.dev/tgrvekc-atlanta-concerts-2024-events-this-weekend-photos-ibhnqxk/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.6_fMiPMrfeqD9K0vgkdR6QHaE8\u0026pid=Api","contentUrl":"https://www.getawaymavens.com/wp-content/uploads/2019/01/Westin-Peachtree-Atlanta.jpg","hostPageUrl":"https://www.getawaymavens.com/downtown-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.a3twsUq07PJg2rb4inuF9gHaD5\u0026pid=Api","contentUrl":"https://www.accessatlanta.com/resizer/Ajxge91STFdTbwY8na01vn0-vx0=/1280x673/smart/cloudfront-us-east-1.images.arcpublishing.com/ajc/OC7R354JWJDEBDM4STAQSVACBY.jpg","hostPageUrl":"https://www.accessatlanta.com/atlanta-things-to-do/10-things-to-do-in-atlanta-this-weekend/RX2FZ7SOE5GF5LOYLPDLNGSAGE/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.Xx-1U0cx62o3LJ01YsvDPwHaHa\u0026pid=Api","contentUrl":"https://unexpectedatlanta.com/wp-content/uploads/2022/12/Fun-Things-to-Do-in-Atlanta-for-Adults.png","hostPageUrl":"https://unexpectedatlanta.com/fun-things-to-do-in-atlanta-for-adults/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.TAb-J4vHX4UsIX--2psZXwHaDt\u0026pid=Api","contentUrl":"https://www.familyvacationcritic.com/wp-content/uploads/sites/19/2019/11/atlanta-centennial-park-hero.jpg","hostPageUrl":"https://www.familyvacationcritic.com/free-things-to-do-in-atlanta-with-kids/art/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.kPDWhtaLIie8olnqL6TkbwHaLH\u0026pid=Api","contentUrl":"https://i.pinimg.com/736x/0d/dc/90/0ddc901f30fb177f17170a916df1ccb6.jpg","hostPageUrl":"https://www.pinterest.com/pin/15-fun-things-to-do-atlanta-georgia-with-your-kids--185351340900142849/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.VX9GFXDDPaUhlk5Ajz2jFAHaLH\u0026pid=Api","contentUrl":"https://www.busytourist.com/wp-content/uploads/2020/07/best-things-to-do-in-Atlanta-GA-1-2.jpg","hostPageUrl":"https://www.busytourist.com/things-to-do-in-atlanta-ga/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.zDsChx3osL1eGz8a5zpM-AHaLH\u0026pid=Api","contentUrl":"https://localadventurer.com/wp-content/uploads/2016/04/free-things-to-do-in-atlanta.jpg","hostPageUrl":"https://localadventurer.com/free-things-to-do-in-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.IqznTH1ocAOkn-XVKO4tyQHaL5\u0026pid=Api","contentUrl":"https://i.pinimg.com/originals/f5/05/8e/f5058e44214459ab6506002aee8aa99f.png","hostPageUrl":"https://www.artofit.org/image-gallery/34480753388071714/101-best-things-to-do-in-atlanta-georgia-the-ultimate-attractions-guide/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.DqkGfBvo28CxyBIESXwj0QHaE8\u0026pid=Api","contentUrl":"https://southerntrippers.com/wp-content/uploads/2022/08/things-to-do-in-atlanta-at-night-.jpg","hostPageUrl":"https://southerntrippers.com/things-to-do-in-atlanta-at-night/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.dOF_E14DjBhURGyO3gpUPAHaE8\u0026pid=Api","contentUrl":"https://www.travelandleisure.com/thmb/vWy4TaS7bYto1vVS6vmZsrxZzY0=/1800x1200/filters:fill(auto,1)/atlanta-skyline-TODOATL0122-195afa04632e43f5be27ef2dcc94ca3b.jpg","hostPageUrl":"https://www.travelandleisure.com/attractions/best-things-to-do-in-atlanta-georgia"}]}}
```

I called the tool named crawl with these arguments:
```
{"q": "things to do in Atlanta January 25-26 2025", "textFormat": "Raw"}
```
and got this result:
```
{"queryContext":{"originalQuery":"things to do in Atlanta January 25-26 2025"},"webPages":{"value":[{"name":"Atlanta Events \u2013 January 2025 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-january.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-january.html","snippet":"Events From January 21 To The Weekend Of January 25-26 2025. \u2192 For concerts this week, see concerts in Atlanta in January \u2013 Love\u2019s Labour\u2019s Lost, January 11-26 2025 Play at the Shakespeare Tavern Playhouse. More details \u2013 Gimme Please!, January 16 - February 16 2025 Children\u2019s play at Alliance Theatre. More details"},{"name":"Atlanta Events January 2025: Concerts, Shows, Sports...","url":"https://atlanta-ga.events/january/","displayUrl":"https://atlanta-ga.events/january","snippet":"Explore our comprehensive calendar for all the current and upcoming events in Atlanta, GA, for January 2025. This guide is your gateway to discovering the best things to do in Atlanta, from Concerts and Theater Shows to Sports events and music festivals. You\u2019ll find a detailed list of all the most engaging and exciting activities happening throughout the month.","thumbnailUrl":"https://www.bing.com/th?id=OIP.ytkv04WKv8RTyYo4_DsP4gHaHa\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"25 Reasons to Visit Atlanta in 2025","url":"https://discoveratlanta.com/stories/things-to-do/25-reasons-to-visit-atlanta-in-2025/","displayUrl":"https://discoveratlanta.com/stories/things-to-do/25-reasons-to-visit-atlanta-in-2025","snippet":"The Atlanta Chinese Dance Company performs on stage on Jan. 25,2025, at Aurora Children\u2019s Playhouse. Here are more ways to celebrate Chinese New Year in Atlanta. Spring Festivals. Atlanta Film Festival 2025 takes place April 24 through May 4, 2025, at the Plaza Theatre \u0026 Tara Theatre."},{"name":"Cool Things To Do in January in Atlanta","url":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-january-in-atlanta/","displayUrl":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-january-in-atlanta","snippet":"Moon Zooz: Too Many Zooz \u002B Moon Hooch with Honeycomb, Jan. 25. MJ Lenderman \u0026 The Wind, Jan. 26-27. James McMurtry, Jan. 29. Feeling Sporty? ATLANTA, GA \u2013 JANUARY 26: Trae Young #11 of the Atlanta Hawks drives to the basket against the LA Clippers on January 26, 2021 at State Farm Arena in Atlanta, Georgia.","thumbnailUrl":"https://www.bing.com/th?id=OIP.aU9EcatRnz4MDkIKyYbkQgHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta January 2025: Events, Concerts, Clubs \u0026 Things to Do - Experism","url":"https://www.experism.com/atlanta/january","displayUrl":"https://www.experism.com/atlanta/january","snippet":"Don\u2019t miss the Atlanta\u0027s 2025 Chick-fil-A Peach Bowl on January 1. As one of college football\u2019s premier postseason games, the Peach Bowl brings together two of the nation\u2019s top teams for a gridiron showdown at Mercedes-Benz Stadium. The Peach Bowl experience includes pre-game festivities, live entertainment, and interactive fan zones.","thumbnailUrl":"https://www.bing.com/th?id=OIP.WKXWyOtbF2Q-oMmg6Y-HiQHaE8\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Fun, Free Activities for Kids in January 2025: 15 Free Things To Do in ...","url":"https://www.msn.com/en-us/sports/other/fun-free-activities-for-kids-in-january-2025-15-free-things-to-do-in-atlanta-with-kids/ar-AA1mOEey","displayUrl":"https://www.msn.com/en-us/sports/other/fun-free-activities-for-kids-in-january-2025-15...","snippet":"Free Things to Do with Kids in Atlanta January 2025. Chick-fil-A Peach Bowl Fan Fest ... January 25, 2025. Atlanta Motor Speedway is inviting fans to come see the historic speedway for themselves ..."},{"name":"Calendar of Events | Explore Georgia","url":"https://exploregeorgia.org/calendar-of-events","displayUrl":"https://exploregeorgia.org/calendar-of-events","snippet":"Zoo Atlanta. View all dates. Atlanta. Monday, December 30. December 30, 2024 - January 31, 2025 Turtles of Georgia. Cloudland Canyon State Park. View all dates. ... January 26, 2025 Let\u0027s ROCK: Geology Hike. Cloudland Canyon State Park. View all dates. Rising Fawn. Showing 1 - 10 of 593 Pagination. Current page 1; Page 2; Page 3 ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.h9jwkN9bKQKns2d4nxZSHgHaE8\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Explore Atlanta Events for Today, This Week or Weekend","url":"https://discoveratlanta.com/events/main/","displayUrl":"https://discoveratlanta.com/events","snippet":"Explore the Atlanta events calendar for things to do today, this week or this weekend. Find upcoming festivals, seasonal events and more! ... Atlanta Black Expo 2025 2/22/25 \u2013 2/23/25. QS Discover Master\u2019s and MBA Fair in Atlanta 2/22/25. ... January 26. Sunday. Georgia Tech Women\u2019s Basketball vs Duke 1/26/25. January 28.","thumbnailUrl":"https://www.bing.com/th?id=OIP.FkxSV6JbrWatsFMM3_8dIAHaEh\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta Concerts in January 2025 - Eventworld","url":"https://www.eventworld.co/atlanta/january/","displayUrl":"https://www.eventworld.co/atlanta/january","snippet":"Find upcoming concerts, gigs \u0026 tour dates in Atlanta, GA in january 2025. ... January 26, 2025: Cobb Energy Performing Arts Centre : $50. Buy tickets. Concerts. Sonic Symphony ... January 25, 2025: Robert Ferst Center for the Arts - Georgia Tech : $90. Buy tickets. Concerts. Silly Goose ..."},{"name":"Atlanta in January | Concerts, Festivals, Tickets \u0026 Tour Dates - Songkick","url":"https://www.songkick.com/metro-areas/4120-us-atlanta/january-2025","displayUrl":"https://www.songkick.com/metro-areas/4120-us-atlanta/january-2025","snippet":"Concerts in Atlanta in January. Find tickets to all live music, concerts, tour dates and festivals in and around Atlanta. Currently there are 64 upcoming events. Filter by artist. ... Sat 25 Jan 2025 Buckhead Theatre Atlanta, GA, US. Cool Company. Fri 22 Nov 2024 The Masquerade - Purgatory Atlanta, GA, US. pigeon pit. Mon 28 ..."}]},"images":{"value":[{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.WoE56TfoQ63T7_SpwvmGKgHaE8\u0026pid=Api","contentUrl":"https://www.experism.com/explore/wp-content/uploads/2023/02/Atlanta-1550x1034.jpg","hostPageUrl":"https://www.experism.com/atlanta/january"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.-1BVEwbN81Oc9jxAOaX2SQHaE5\u0026pid=Api","contentUrl":"https://southerntrippers.com/wp-content/uploads/2022/08/things-to-do-in-atlanta-at-night-centennial-olympic-park-.jpg","hostPageUrl":"https://nertqherminia.pages.dev/yfskiot-things-to-do-in-atlanta-georgia-in-january-2024-wwtcyrq/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.5jERbhhnl8LiECUQySoRYAHaG1\u0026pid=Api","contentUrl":"https://uphomes.com/storage/blogposts/February2023/medium/RKuDJcqErH176HD3LyWV.png","hostPageUrl":"https://uphomes.com/blog/things-to-do-atlanta-ga/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.k7hGzw27Lx3uqX8C1eseEQHaE6\u0026pid=Api","contentUrl":"https://365atlantatraveler.com/wp-content/uploads/2022/05/atlanta-1280x850.jpeg","hostPageUrl":"https://nertqherminia.pages.dev/yfskiot-things-to-do-in-atlanta-georgia-in-january-2024-wwtcyrq/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.z9bHreP8_hqtZ1wfgtW3NQHaE8\u0026pid=Api","contentUrl":"https://lp-cms-production.imgix.net/2021-05/GettyRF_486068945.jpg?auto=format\u0026q=40\u0026w=870\u0026dpr=5","hostPageUrl":"https://www.lonelyplanet.com/articles/best-parks-atlanta"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.fqwDUYiMaHe0id6KtJYyfwAAAA\u0026pid=Api","contentUrl":"https://unexpectedatlanta.com/wp-content/uploads/2022/12/Visiting-Atlanta-in-January-Ultimate-Guide-1.png","hostPageUrl":"https://unexpectedatlanta.com/atlanta-in-january/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.6_fMiPMrfeqD9K0vgkdR6QHaE8\u0026pid=Api","contentUrl":"https://www.getawaymavens.com/wp-content/uploads/2019/01/Westin-Peachtree-Atlanta.jpg","hostPageUrl":"https://www.getawaymavens.com/downtown-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.F6kNmu6vnzKivPa3ikGUTQHaFj\u0026pid=Api","contentUrl":"https://www.travelalerts.ca/wp-content/uploads/2017/05/Atlanta1600x1200-1-1024x768.jpg","hostPageUrl":"https://nertqherminia.pages.dev/yfskiot-things-to-do-in-atlanta-georgia-in-january-2024-wwtcyrq/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.ENao3rD4rUbnHGA6QJHXYgAAAA\u0026pid=Api","contentUrl":"https://res.cloudinary.com/hz3gmuqw6/image/upload/c_fill,q_auto,w_750/f_auto/things-to-do-in-atlanta-phpWlZ13b","hostPageUrl":"https://www.cozymeal.com/magazine/things-to-do-in-atlanta"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.AsEhWz77ez1CIx9RVZsvFAHaEh\u0026pid=Api","contentUrl":"https://res.cloudinary.com/atlanta/images/v1636654462/newAtlanta.com/Atlanta_OneMusicFest_COP-1/Atlanta_OneMusicFest_COP-1.jpg?_i=AA","hostPageUrl":"https://karylbridgette.pages.dev/tgrvekc-atlanta-concerts-2024-events-this-weekend-photos-ibhnqxk/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.Xx-1U0cx62o3LJ01YsvDPwHaHa\u0026pid=Api","contentUrl":"https://unexpectedatlanta.com/wp-content/uploads/2022/12/Fun-Things-to-Do-in-Atlanta-for-Adults.png","hostPageUrl":"https://unexpectedatlanta.com/fun-things-to-do-in-atlanta-for-adults/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.uXSLRLfe7U8MxUYx6YGGVAHaE7\u0026pid=Api","contentUrl":"https://quartzmountain.org/images/resources/things-to-do-atlanta-january_20230726190854.webp","hostPageUrl":"https://quartzmountain.org/article/things-to-do-atlanta-january"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.TAb-J4vHX4UsIX--2psZXwHaDt\u0026pid=Api","contentUrl":"https://www.familyvacationcritic.com/wp-content/uploads/sites/19/2019/11/atlanta-centennial-park-hero.jpg?is-pending-load=1","hostPageUrl":"https://www.familyvacationcritic.com/free-things-to-do-in-atlanta-with-kids/art/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.VX9GFXDDPaUhlk5Ajz2jFAHaLH\u0026pid=Api","contentUrl":"https://www.busytourist.com/wp-content/uploads/2020/07/best-things-to-do-in-Atlanta-GA-1-2.jpg","hostPageUrl":"https://www.busytourist.com/things-to-do-in-atlanta-ga/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.kPDWhtaLIie8olnqL6TkbwHaLH\u0026pid=Api","contentUrl":"https://i.pinimg.com/736x/0d/dc/90/0ddc901f30fb177f17170a916df1ccb6.jpg","hostPageUrl":"https://www.pinterest.com/pin/15-fun-things-to-do-atlanta-georgia-with-your-kids--185351340900142849/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.IqznTH1ocAOkn-XVKO4tyQHaL5\u0026pid=Api","contentUrl":"https://i.pinimg.com/originals/f5/05/8e/f5058e44214459ab6506002aee8aa99f.png","hostPageUrl":"https://www.artofit.org/image-gallery/34480753388071714/101-best-things-to-do-in-atlanta-georgia-the-ultimate-attractions-guide/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.zDsChx3osL1eGz8a5zpM-AHaLH\u0026pid=Api","contentUrl":"https://localadventurer.com/wp-content/uploads/2016/04/free-things-to-do-in-atlanta.jpg","hostPageUrl":"https://localadventurer.com/free-things-to-do-in-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.dOF_E14DjBhURGyO3gpUPAHaE8\u0026pid=Api","contentUrl":"https://www.travelandleisure.com/thmb/vWy4TaS7bYto1vVS6vmZsrxZzY0=/1800x1200/filters:fill(auto,1)/atlanta-skyline-TODOATL0122-195afa04632e43f5be27ef2dcc94ca3b.jpg","hostPageUrl":"https://www.travelandleisure.com/attractions/best-things-to-do-in-atlanta-georgia"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.kxVeVo4GNFZJwN_RXwU1AwHaGP\u0026pid=Api","contentUrl":"https://infographicjournal.com/wp-content/uploads/2019/12/Things-to-Do-Atlanta-feat.jpg","hostPageUrl":"https://infographicjournal.com/top-50-things-to-do-in-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.v6IPB_NTvbmy-pxyZSCA0QHaE7\u0026pid=Api","contentUrl":"http://rwethereyetmom.com/wp-content/uploads/2015/08/6-Things-to-do-in-Atlanta-1024x681.jpg","hostPageUrl":"http://rwethereyetmom.com/6-things-to-do-in-atlanta.html"}]}}
```

I called the tool named ExtractContentFromUrl with these arguments:
```
{"assistantName": "WebPageAgentAnt", "instructions": "Find events happening on January 4-5, 2025 in Atlanta. https://atlanta-ga.events/january/"}
```
and got this result:
```
"Here are the events happening in Atlanta on January 4-5, 2025:\n\n### January 4, 2025\n\n1. **Titanic: An Immersive Voyage**\n   - **Location:** Exhibition Hub Art Center Atlanta, 5660 Buford Hwy NE, Atlanta, GA\n   - **Price:** From $64\n   - **Tickets Remaining:** 32\n   - [Tickets](https://atlanta-ga.events/event/titanic-an-immersive-voyage-atlanta-saturday-january-4-2025-tba/?pid=8451)\n\n2. **Georgia Tech Yellow Jackets vs. Boston College Eagles**\n   - **Location:** McCamish Pavilion, 965 Fowler St., Atlanta, GA\n   - **Time:** 12:00 PM\n   - **Price:** From $8\n   - **Available Tickets:** 336\n   - ![Georgia Tech Yellow Jackets vs. Boston College Eagles](https://atlanta-ga.events/wp-content/uploads/2023/08/Georgia-Tech-Yellow-Jackets-Basketball.jpg)\n   - [Tickets](https://atlanta-ga.events/event/georgia-tech-yellow-jackets-vs-boston-college-eagles-atlanta-saturday-january-4-2025-1200-pm/?pid=1282)\n\n3. **SalgirahArts Festival: Expressions of Shukran**\n   - **Location:** Gas South Theater, 6400 Sugarloaf Parkway, Duluth, GA\n   - **Time:** 12:00 PM \u0026 8:00 PM\n   - **Price:** From $22\n   - **Available Tickets:** 211\n   - [Tickets](https://atlanta-ga.events/event/salgiraharts-festival-expressions-of-shukran-duluth-saturday-january-4-2025-1200-pm/?pid=8220)\n\n4. **Kennesaw State Owls vs. Jacksonville State Gamecocks**\n   - **Location:** KSU Convocation Center, 590 Cobb Ave., Kennesaw, GA\n   - **Time:** 5:00 PM\n   - **Price:** From $25\n   - **Tickets Remaining:** 35\n   - ![Kennesaw State Owls vs. Jacksonville State Gamecocks](https://atlanta-ga.events/wp-content/uploads/2024/02/kennesaw-state-owls-basketball.jpg)\n   - [Tickets](https://atlanta-ga.events/event/kennesaw-state-owls-vs-jacksonville-state-gamecocks-kennesaw-saturday-january-4-2025-500-pm/?pid=4812)\n\n5. **Georgia State Panthers vs. Louisiana-Lafayette Ragin\u0027 Cajuns**\n   - **Location:** Georgia State Convocation Center, 455 Capitol Ave SE, Atlanta, GA\n   - **Time:** 7:00 PM\n   - **Price:** From $10\n   - **Tickets Remaining:** 46\n   - ![Georgia State Panthers vs. Louisiana-Lafayette Ragin\u0027 Cajuns](https://atlanta-ga.events/wp-content/uploads/2023/08/Georgia-State-Panthers-Basketball.jpg)\n   - [Tickets](https://atlanta-ga.events/event/georgia-state-panthers-vs-louisiana-lafayette-ragin-cajuns-atlanta-saturday-january-4-2025-700-pm/?pid=1302)\n\n6. **Canopy**\n   - **Location:** Altar at Masquerade - GA, 75 MLK Jr Dr SW, Atlanta, GA\n   - **Time:** 7:00 PM\n   - **Price:** From $23\n   - **Tickets Remaining:** 18\n   - [Tickets](https://atlanta-ga.events/event/canopy-atlanta-saturday-january-4-2025-700-pm/?pid=8191)\n\n7. **Kaya\u0027s Embrace**\n   - **Location:** Purgatory Stage at Masquerade - GA, 75 MLK Jr Dr SW, Atlanta, GA\n   - **Time:** 7:00 PM\n   - **Price:** From $25\n   - **Tickets Remaining:** 42\n   - [Tickets](https://atlanta-ga.events/event/kayas-embrace-atlanta-saturday-january-4-2025-700-pm/?pid=8401)\n\n8. **Da\u0027Vinchi**\n   - **Location:** ATL Comedy Theater, 4650 Jimmy Carter Boulevard, Norcross, GA\n   - **Time:** 7:00 PM\n   - **Price:** From $59\n   - **Available Tickets:** 60\n   - [Tickets](https://atlanta-ga.events/event/davinchi-norcross-saturday-january-4-2025-700-pm/?pid=8304)\n\n9. **Georgia Swarm vs. Rochester Knighthawks**\n   - **Location:** Gas South Arena, 6400 Sugarloaf Parkway, Duluth, GA\n   - **Time:** 7:30 PM\n   - **Price:** From $9\n   - **Available Tickets:** 507\n   - ![Georgia Swarm vs. Rochester Knighthawks](https://atlanta-ga.events/wp-content/uploads/2024/02/georgia-swarm.jpg)\n   - [Tickets](https://atlanta-ga.events/event/georgia-swarm-vs-rochester-knighthawks-duluth-saturday-january-4-2025-730-pm/?pid=5028)\n\n10. **Sonic Symphony Live**\n    - **Location:** Cobb Energy Performing Arts Centre, 2800 Cobb Galleria Pkwy, Atlanta, GA\n    - **Time:** 8:00 PM\n    - **Price:** From $125\n    - **Available Tickets:** 59\n    - ![Sonic Symphony Live](https://atlanta-ga.events/wp-content/uploads/2023/08/Sonic-Symphony-Live.jpg)\n    - [Tickets](https://atlanta-ga.events/event/sonic-symphony-live-atlanta-saturday-january-4-2025-800-pm/?pid=4109)\n\n11. **Best of Atlanta Comedy Showcase**\n    - **Location:** Helium Comedy Club - Atlanta, 3070 Windward Plaza, Alpharetta, GA\n    - **Time:** 8:30 PM\n    - **Price:** From $24\n    - **Tickets Remaining:** 32\n    - ![Best of Atlanta Comedy Showcase](https://atlanta-ga.events/wp-content/uploads/2024/12/best-of-atlanta-comedy-showcase.jpeg)\n    - [Tickets](https://atlanta-ga.events/event/best-of-atlanta-comedy-showcase-alpharetta-saturday-january-4-2025-830-pm/?pid=2327)\n\n12. **Club 90s: Justin Bieber Night**\n    - **Location:** Buckhead Theatre, 3110 Roswell Road, Atlanta, GA\n    - **Time:** 9:00 PM\n    - **Price:** From $34\n    - **Tickets Remaining:** 30\n    - ![Club 90s: Justin Bieber Night](https://atlanta-ga.events/wp-content/uploads/2024/02/club-90s.jpg)\n    - [Tickets](https://atlanta-ga.events/event/club-90s-justin-bieber-night-atlanta-saturday-january-4-2025-900-pm/?pid=8384)\n\n### January 5, 2025\n\nThe website does not list specific events for January 5, 2025."
```

I called the tool named ExtractContentFromUrl with these arguments:
```
{"assistantName": "WebPageAgentAnt", "instructions": "Find events happening on January 11-12, 2025 in Atlanta. https://www.experism.com/atlanta/january"}
```
and got this result:
```
"Here are events happening in Atlanta on January 11-12, 2025:\n\n### Sarah Silverman\n- **Date:** January 11, 2025\n- **Description:** Don\u2019t miss two-time Emmy Award-winning comedian Sarah Silverman and her latest hilarious takes on the modern experience.\n- **[Reserve your seats](https://tickets.experism.com/search?q=sarah%2Bsilverman%2Bgeorgia)**\n- **[View on Map](https://www.stay22.com/embed/gm?aid=experism\u0026lat=33.88471\u0026lng=-84.4585962\u0026checkin=2025-01-11)**\n\n![](https://img.youtube.com/vi/Nmc7h0guBJI/maxresdefault.jpg)\n\nFor more information and to check other events, you can visit the [Experism Atlanta January page](https://www.experism.com/atlanta/january)."
```

I called the tool named ExtractContentFromUrl with these arguments:
```
{"assistantName": "WebPageAgentAnt", "instructions": "Find events happening on January 18-19, 2025 in Atlanta. https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-january-in-atlanta/"}
```
and got this result:
```
"Here are some events happening in Atlanta on January 18-19, 2025:\n\n### Shows and Concerts\n- **Music for the Very Young** \u2013 The Lion and the Mouse: A Musical Tale for Little Ears, Jan. 18-19\n- **SWAG Gospel**, Jan. 19\n- **Lil Wayne with GloRilla \u0026 Muni Long**, Jan. 18 at State Farm Arena\n- **Camila Cabello with Myles Smith \u0026 Knox**, Jan. 19 at State Farm Arena\n\n### Sporting Events\n- **Atlanta Vibe vs. Grand Rapids Rise**, Jan. 19 at 3 p.m.\n\n### Special Events\n- **IllumiNights at the Zoo** \u2013 Enjoy beautiful lanterns and a nighttime wonderland through Jan. 19.\n- **Giants: Art from the Dean Collection of Swizz Beatz and Alicia Keys** \u2013 An art exhibit running through Jan. 19 at the High Museum of Art.\n- **Playoff Fan Central** \u2013 Part of the College Football Playoff National Championship festivities, open Jan. 18-20 from 10 a.m. to 6 p.m. at the Georgia World Congress Center Building A.\n- **Taste of the Championship** \u2013 A gourmet dining experience at the Georgia Aquarium on Jan. 19.\n\n### College Football Playoff Events\n- **AT\u0026T Playoff Playlist Live!** \u2013 Featuring performances by Lil Wayne, GloRilla, and Muni Long on Jan. 18 and Camila Cabello, Myles Smith, and Knox on Jan. 19.\n- **Trophy Trot** \u2013 The official road race of the College Football Playoff on Jan. 19.\n\nThese events offer a range of entertainment options including music, sports, and cultural experiences."
```

I called the tool named ExtractContentFromUrl with these arguments:
```
{"assistantName": "WebPageAgentAnt", "instructions": "Find events happening on January 25-26, 2025 in Atlanta. https://discoveratlanta.com/stories/things-to-do/25-reasons-to-visit-atlanta-in-2025/"}
```
and got this result:
```
"Here are some events happening in Atlanta on January 25-26, 2025:\n\n1. **Lunar New Year Celebrations:**\n   - **Lunar New Year at Stone Mountain Park**: Part of several events celebrating the Year of the Snake from January 25 through February 9, 2025.\n   - **Atlanta Chinese Dance Company Performance**: Performing on stage on January 25, 2025, at Aurora Children\u2019s Playhouse.\n\n   More details about the Lunar New Year celebrations can be found [here](https://discoveratlanta.com/event/detail/lunar-new-year-celebration/).\n\nThese events are part of a broader celebration and cultural offerings in Atlanta. For more information, you might want to check local event calendars closer to the date."
```

Here are some exciting events happening in Atlanta over the next four weekends:

### January 4-5, 2025
1. **Titanic: An Immersive Voyage**
   - **Location:** Exhibition Hub Art Center Atlanta, 5660 Buford Hwy NE, Atlanta, GA
   - **Price:** From $64
   - [More Info & Tickets](https://atlanta-ga.events/event/titanic-an-immersive-voyage-atlanta-saturday-january-4-2025-tba/?pid=8451)

2. **Georgia Tech Yellow Jackets vs. Boston College Eagles**
   - **Location:** McCamish Pavilion, 965 Fowler St., Atlanta, GA
   - **Time:** 12:00 PM
   - **Price:** From $8
   - ![Event Image](https://atlanta-ga.events/wp-content/uploads/2023/08/Georgia-Tech-Yellow-Jackets-Basketball.jpg)
   - [More Info & Tickets](https://atlanta-ga.events/event/georgia-tech-yellow-jackets-vs-boston-college-eagles-atlanta-saturday-january-4-2025-1200-pm/?pid=1282)

3. **Sonic Symphony Live**
   - **Location:** Cobb Energy Performing Arts Centre, 2800 Cobb Galleria Pkwy, Atlanta, GA
   - **Time:** 8:00 PM
   - **Price:** From $125
   - ![Event Image](https://atlanta-ga.events/wp-content/uploads/2023/08/Sonic-Symphony-Live.jpg)
   - [More Info & Tickets](https://atlanta-ga.events/event/sonic-symphony-live-atlanta-saturday-january-4-2025-800-pm/?pid=4109)

For more events, visit [Atlanta GA Events](https://atlanta-ga.events/january/).

### January 11-12, 2025
1. **Sarah Silverman Live**
   - **Date:** January 11, 2025
   - **Description:** Enjoy a night of comedy with two-time Emmy Award-winning comedian Sarah Silverman.
   - ![Event Image](https://img.youtube.com/vi/Nmc7h0guBJI/maxresdefault.jpg)
   - [Reserve your seats](https://tickets.experism.com/search?q=sarah%2Bsilverman%2Bgeorgia)

For more events, check out [Experism Atlanta January](https://www.experism.com/atlanta/january).

### January 18-19, 2025
1. **Lil Wayne with GloRilla & Muni Long**
   - **Date:** January 18
   - **Location:** State Farm Arena
2. **Camila Cabello with Myles Smith & Knox**
   - **Date:** January 19
   - **Location:** State Farm Arena
3. **IllumiNights at the Zoo**
   - **Date:** Through January 19
   - Enjoy beautiful lantern displays at Zoo Atlanta.

For a full list of events, visit [Discover Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-january-in-atlanta/).

### January 25-26, 2025
1. **Lunar New Year at Stone Mountain Park**
   - Celebrate the Year of the Snake with various activities from January 25 through February 9, 2025.
2. **Atlanta Chinese Dance Company Performance**
   - **Date:** January 25, 2025
   - **Location:** Aurora Children’s Playhouse

For more details, check out [Discover Atlanta](https://discoveratlanta.com/stories/things-to-do/25-reasons-to-visit-atlanta-in-2025/).

These events offer a wide range of entertainment options, including music, sports, cultural experiences, and more!


## Clean Up

In [7]:
var assistant = (await client.AssistantList())?.Data?.FirstOrDefault(o => o.Name == "WebSearchAgentAnt");
if(assistant != null) {
    await client.AssistantDelete(assistant.Id);
    Console.WriteLine("Deleted assistant");
}
else
{
    Console.WriteLine("Didn't find WebSearchAgentAnt");
}

assistant = (await client.AssistantList())?.Data?.FirstOrDefault(o => o.Name == "WebPageAgentAnt");
if(assistant != null) {
    await client.AssistantDelete(assistant.Id);
    Console.WriteLine("Deleted assistant");
}
else
{
    Console.WriteLine("Didn't find WebPageAgentAnt");
}

Deleted assistant
Deleted assistant
